## init

import needed packages

In [18]:
import os,json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import numpy as np
# remove avx warning:
# I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
# To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

from pandas import DataFrame,Series
from usefull_functions_new import random_list


load test data (if not loaded from outside)

In [19]:
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False



try: use_MASTER
except: use_MASTER = False

if not use_MASTER:
    #%run imports.ipynb
    from pathlib import Path
    from sys import path
    
    #pickle,sys
    home = str(Path.home())
    save_add = f'{home}/Dropbox/CyTOF_Breast/data_yishai/'
    
    #add the path were the functions are and load the small dataset (used for testing) and config data
    path.append(f'{home}/Desktop/breast_cancer_PHD_research/functions/')
    # with open(f'{save_add}/Dropbox/CyTOF_Breast/data_yishai/default_data.p', 'rb') as f:
    #   [df,config] = pickle.load(f)
    with open(f'{save_add}default_sample.json', "r") as f:
        df = DataFrame.from_dict(json.load(f)).reset_index(drop=True)
    with open(f'{save_add}default_config.json', "r") as f:
        config = json.load(f) 
    del f, home,save_add
    




import needed packages

In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import numpy as np
# remove avx warning:
# I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
# To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

from pandas import DataFrame,Series

from plot_functions_new import class_colors

%matplotlib inline

define classification class

In [21]:

from metrics_and_dataset_utils import *  
from xg_shap import Shap, fitXGBClassifier


get labels for classification

In [22]:
if not use_MASTER:
    samples = Series(df['samp'])
    # .strdrop_duplicates()
    
    LEN = int(df.shape[0]/2)
    labels = Series(np.random.randint(-1,4,LEN),index = df.index[:LEN])#random labels for testing
    
    # the labels of each single sample clustering
    # sample_labels = Series(np.random.randint(-1,5,df.shape[0]),index = df.index)#random labels for testing
    # sample_labels.loc[0],samples.loc[0]=-99,4.0#add error to test error handling
    classes = Series(random_list(arr = ['Unknown', 'Noise' ,'Luminal' ,'Basal-like' ,'Cycling'],LEN = LEN ),index = df.index[:LEN])
    


split into the train-test datasets

In [23]:
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),labels.copy())
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),labels.copy(),samples.copy(),train_samples = [4,7,8,14],test_samples = [  5,11,13,15,17,18,19,20])
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),labels.copy(),samples.copy(),train_samples = [4],test_samples = [  4.1])

train-test data split using random split
train-test data split using sample indexes: train-[4, 7, 8, 14],test-[5, 11, 13, 15, 17, 18, 19, 20]
train-test data split using sample indexes: train-[4],test-[4.1]


## XGBClassifier fit, calculate metrics & shap

create new classes: 
1. classes - ['Unknown' 'Noise' 'Cycling' 'Luminal' 'Basal-like']
2. classes1 - ['Cycling' 'Lum&Basal']
3. xlasses2 - ['Luminal' 'Basal-like']

In [24]:
classes1 = classes[(classes=='Basal-like')|(classes=='Luminal')|(classes=='Cycling')].copy()
classes1[(classes=='Basal-like')|(classes=='Luminal')] = 'Lum&Basal'
classes2 = classes[(classes=='Basal-like')|(classes=='Luminal')].copy()
print(sorted(classes.unique()))
print(sorted(classes1.unique()))
print(sorted(classes2.unique()))


['Basal-like', 'Cycling', 'Luminal', 'Noise', 'Unknown']
['Cycling', 'Lum&Basal']
['Basal-like', 'Luminal']


original classes (all 3 & noise and unknown); XGBClassifier fit, calculate metrics & shap

In [ ]:

# drop df rows without label (drop points without label) and split randomly
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes.copy())

model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

# predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                .plot_all(config['dir_plots'] +'5_'+ config['figname']+'using_'+'all_classes_')

Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                         figname = '5_Shap_' + config['figname']+'using_'+'all_classes_')
print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")


Lum&Basal_vs_Cycling


In [ ]:

# drop df rows without label (drop points without label) and split randomly
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes1.copy())

model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

# predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                .plot_all(config['dir_plots'] + '5_'+config['figname']+'using_'+'Lum&Basal_vs_Cycling_')

Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                         figname = '5_Shap_' + config['figname']+'using_'+'Lum&Basal_vs_Cycling_')
print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")


Lum_vs_Basal

In [ ]:

# drop df rows without label (drop points without label) and split randomly
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes2.copy())

model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

# predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                .plot_all(config['dir_plots'] + '5_'+config['figname']+'using_'+'Lum_vs_Basal_')

Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                         figname = '5_Shap_' + config['figname']+'using_'+'Lum_vs_Basal_')
print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")
